In [15]:
import random
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)

reviews_file = open('reviews.txt')
raw_reviews = reviews_file.readlines()
reviews_file.close()

tokens = list(map(lambda x: x.split(" "), raw_reviews))
wordCount = Counter()
for sentence in tokens:
    for word in sentence:
        wordCount[word] -= 1
vocab = list(set(map(lambda x: x[0], wordCount.most_common())))

word_to_index = {}
for i, word in enumerate(vocab):
    word_to_index[word] = i

concatenated = list()
input_dataset = list()
for sentence in tokens:
    sentence_indices = list()
    for word in sentence:
        try:
            sentence_indices.append(word_to_index[word])
            concatenated.append(word_to_index[word])
        except:
            ""
    input_dataset.append(sentence_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [16]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha = 0.05
iterations = 2
hidden_size = 50
window = 2
negatives = 5

weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2 # this is the same before
weights_1_2 = np.random.rand(len(vocab), hidden_size) * 0 # can I just set this to zero? Maybe each random depends on its last value

layer_2_target = np.zeros(negatives + 1)
layer_2_target[0] = 1


In [17]:
# gives a list of similar words, according to our model
# Euclidian distance
from collections import Counter
import math

def similar(target='beautiful'):
    target_index = word_to_index[target]
    scores = Counter()
    for word, index in word_to_index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10) 

In [20]:
# Word analogies

def analogy(positives = ['terrible', 'good'], negatives = ['bad']):
    norms = np.sum(weights_0_1 * weights_0_1, axis = 1)
    norms.resize(norms.shape[0], 1)
    
    normed_weights = weights_0_1 * norms
    query_vector = np.zeros(len(weights_0_1[0]))
    for word in positives:
        query_vector += normed_weights[word_to_index[word]]
    for word in negatives:
        query_vector -= normed_weights[word_to_index[word]]

    scores = Counter()
    for word, index in word_to_index.items():
        raw_difference = weights_0_1[index] - query_vector
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)[1:]

In [19]:
import sys
for review_index, review in enumerate(input_dataset * iterations):
    for target_index in range(len(review)):
        target_samples = [review[target_index]] + list(concatenated[(np.random.rand(negatives) * len(concatenated)).astype('int').tolist()]) # hard to read # whats with the stuff in [...] in the beginning?

        left_context = review[max(0, target_index - window) : target_index]
        right_context = review[target_index + 1 : min(len(review), target_index + window)]

        layer_1 = np.mean(weights_0_1[left_context + right_context], axis = 0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))

        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context + right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha

    if(review_index % 250 == 0): 
        sys.stdout.write('\rProgress ' + str(review_index/float(len(input_dataset) * iterations)) + "      " + str(similar('terrible')))
    sys.stdout.write('\rProgress ' + str(review_index/float(len(input_dataset) * iterations)))

Progress 0.99996   [('terrible', -0.0), ('brilliant', -2.9864360353531545), ('superb', -3.2051343193225095), ('horrible', -3.348405222422591), ('fantastic', -3.5511215163387018), ('pathetic', -3.6404823712942846), ('wonderful', -3.7497748582018438), ('terrific', -3.889212300404385), ('lame', -3.9081653424540193), ('breathtaking', -4.018473404772363)])]47)]][('fine', -200.7129350043311), ('great', -200.85303792955642), ('decent', -200.91520825189565), ('terrific', -201.0147843331458), ('perfect', -201.0693647403121), ('terrible', -201.1500098714689), ('taken', -201.18587064724312), ('superb', -201.21116657741564), ('wonderful', -201.22003039945577)]


In [25]:
print(analogy(['queen', 'man'],['woman'])) # doesnt seem to work ):

[('haven', -166.57992871729513), ('couldn', -166.68016604672235), ('n', -166.9308556580061), ('wouldn', -166.99690894873476), ('de', -167.07565651615565), ('performances', -167.1129641676009), ('ain', -167.17422560702866), ('hasn', -167.3577842029716), ('didn', -167.37077851238914)]
